In [3]:

import shapefile
import shapely
from rtree import index
from shapely.geometry import Polygon
from shapely.geometry import Point, Polygon, MultiPolygon, mapping

"""
Class to calculate if a point is within a polygon

"""
class polygon_util:

    polygons = []
    idx = index.Index()
    
    def __init__(self, sf):
        
        #Load the shapefile of polygons and convert it to shapely polygon objects
    
        polygons_sf = shapefile.Reader(sf)
        polygon_shapes = polygons_sf.shapes()
        polygon_points = [q.points for q in polygon_shapes ]
        self.polygons = [Polygon(q) for q in polygon_points]
        
        #Build a spatial index based on the bounding boxes of the polygon      
        count = -1
        for q in polygon_shapes:
            count +=1
            self.idx.insert(count, q.bbox)

        

    def find_polygons(self):
        return self.idx, self.polygons

    def find_polygonid(self, points):

        #Assign one or more matching polygons to each point
        matches = []
        for i in range(len(points)): #Iterate through each point
            temp= 0
            #Iterate only through the bounding boxes which contain the point
            for j in self.idx.intersection(points[i].coords[0]):
                #Verify that point is within the polygon itself not just the bounding box
                if points[i].within(self.polygons[j]):
                    #print("Match found! ",j)
                    temp=j+1
                    break
            matches.append(temp) #Either the first match found, or None for no matches

        return matches
    
    def find_polygon_latlong(self, long, lat):
        
        point = Point(long, lat)
        #Assign one or more matching polygons to each point
        matches = []
        temp = 0
        
        #Iterate only through the bounding boxes which contain the point
        for j in self.idx.intersection(point.coords[0]):
            #Verify that point is within the polygon itself not just the bounding box
            if point.within(self.polygons[j]):
                #print("Match found! ",j)
                temp=j+1
                break
     
        return temp


    
    def find_polygonid(self, point):


        #Assign one or more matching polygons to each point
        matches = []
        temp = None
        
        #Iterate only through the bounding boxes which contain the point
        for j in self.idx.intersection(point.coords[0]):
            #Verify that point is within the polygon itself not just the bounding box
            if point.within(self.polygons[j]):
                #print("Match found! ",j)
                temp=j+1
                break
     
        return temp





In [9]:
import csv
import shapely
import fiona
from shapely.geometry import Point, Polygon, MultiPolygon, mapping
from fiona import collection
from collections import namedtuple


def generate_rows(file):
    n = 0
    fields = ("medallion","hack_license","vendor_id","rate_code","store_and_fwd_flag","pickup_datetime","dropoff_datetime","passenger_count","trip_time_in_secs","trip_distance","pickup_longitude","pickup_latitude","dropoff_longitude","dropoff_latitude")
    #newfields = ("hack_license","pickup_datetime","dropoff_datetime","passenger_count","trip_time_in_secs","trip_distance","pickup_longitude","pickup_latitude","dropoff_longitude","dropoff_latitude")
    pol = polygon_util("taxi_zone_WGS84.shp")
    ride = namedtuple("ride", fields)
    
    with open(file, 'rt') as f:
            #reader = csv.DictReader(f, delimiter=',')
        try:
            f.readline()
            reader = csv.reader(f)

            #ride = namedtuple('riderecord',next(reader))
            for row in map(ride._make,reader):
                n += 1
                if row.pickup_longitude and row.pickup_latitude and row.dropoff_longitude and row.dropoff_latitude:

                    pu_point = Point(float(row.pickup_longitude), float(row.pickup_latitude))
                    do_point = Point(float(row.dropoff_longitude), float(row.dropoff_latitude))


                    pu_location_id = pol.find_polygonid(pu_point)
                    do_location_id = pol.find_polygonid(do_point)
                    if pu_location_id in (7,236,237,138) or do_location_id in (7,236,237,138):
                        yield row.hack_license,row.pickup_datetime,row.dropoff_datetime,row.passenger_count,row.trip_time_in_secs,row.trip_distance,pu_location_id,do_location_id

                else:
                    continue



        except:
            print(str(n)+"line")
            print(row.hack_license,row.pickup_datetime)
            print(row.hack_license,row.pickup_longitude)
            print(row.hack_license,row.pickup_latitude)
            print(row.hack_license,row.dropoff_longitude)
            print(row.hack_license,row.dropoff_latitude)


filename = ['xaa','xab','xac','xad']
for file in filename:
    with open(file+"_result.csv", "w") as outfile:
        writes = csv.writer(outfile, delimiter=',', quoting=csv.QUOTE_ALL)
        writes.writerows(generate_rows(file))  

print("saved to file")

saved to file
